In [ ]:
import numpy as np
import math
import pandas as pd
from scipy.interpolate import RegularGridInterpolator

バルブのソースコード

In [ ]:
# バルブ特性
class Valve:
    def __init__(self, cv_max=800, r=100):
        # cv_max    :流量係数。
        # r         :レンジアビリティ。最も弁を閉じたときの流量比の逆数
        # vlv       :バルブ開度(1:全開,0:全閉)
        # dp        :バルブによる圧力損失[kPa]
        # g         ：流量 単位要確認!!
        self.cv_max = cv_max
        self.r = r
        self.dp = 0
        self.vlv = 0
        self.g = 0
        
    def f2p(self, g): # flow to pressure
        self.g = g
        # cv = self.cv_max * self.r**(self.vlv - 1)
        # self.dp = - 1743 * (self.g * 1000 / 60)**2 / cv**2 イコールパーセント特性
        if (self.cv_max * self.r**(self.vlv - 1))**2 > 0:
            self.dp = (- 1743 * (1000 / 60)**2 / (self.cv_max * self.r**(self.vlv - 1))**2) * self.g**2
        else:
            self.dp = 0
            
        return self.dp
    
    def p2f(self,dp):
        self.dp = dp
        if self.dp < 0:
            self.g = (self.dp/((- 1743 * (1000 / 60)**2 / (self.cv_max * self.r**(self.vlv - 1))**2)))**0.5
        else:
            self.g = - (self.dp/((1743 * (1000 / 60)**2 / (self.cv_max * self.r**(self.vlv - 1))**2)))**0.5 # 逆流
        
        return self.g
    
    def f2p_co(self): # coefficient for f2p
        return np.array([0,0,(- 1743 * (1000 / 60)**2 / (self.cv_max * self.r**(self.vlv - 1))**2)])
        

バルブのAPIドキュメント

In [ ]:
# Valve.f2p(cv_max=800, r=100, vlv=0, g)
# 流量を用いてバルブにより圧力損失を求める
# パラメータ : cv_max :流量係数
#             r :レンジアビリティ。最も弁を閉じたときの流量比の逆数 
#             vlv :バルブ開度(1:全開,0:全閉)
#             g :流量
# 戻り値: dp :バルブによる圧力損失[kPa]
# Error Returns: dp = 0 


# Valve.p2f(cv_max=800, r=100, vlv=0, dp)
# バルブにより圧力損失を用いて流量を求める
# パラメータ : cv_max :流量係数
#             r :レンジアビリティ。最も弁を閉じたときの流量比の逆数 
#             vlv :バルブ開度(1:全開,0:全閉)
#             dp :バルブによる圧力損失[kPa]
# 戻り値: g :流量[m3/min]


# Valve.f2p_co(cv_max=800, r=100, vlv=0)
# 流量を用いてバルブにより圧力損失を求める時の効率を計算する
# パラメータ : cv_max :流量係数
#             r :レンジアビリティ。最も弁を閉じたときの流量比の逆数 
#             vlv :バルブ開度(1:全開,0:全閉)
# 戻り値: np.array

以下でバルブの設定値を入れる

In [ ]:
valve = Valve()
#以下で初期値を入れる
valve.cv_max = 800
valve.r = 100
valve.vlv = 0.5

In [ ]:
g = 1

In [ ]:
print(valve.f2p(g))

-75.65104166666669


ポンプのソースコード

In [ ]:
# ポンプ特性と消費電力計算
class Pump:
    # 定格値の入力
    def __init__(self, pg=[233,5.9578,-4.95], eg=[0.0099,0.4174,-0.0508], r_ef=0.8):
        # pq    :圧力-流量(pg)曲線の係数（切片、一次、二次）
        # eq    :効率-流量(eg)曲線の係数（切片、一次、二次）
        # r_ef  :定格時の最高効率(本来は計算によって求める？)rated efficiency
        # inv   :回転数比(0.0~1.0)
        # dp_p  :ポンプ揚程[kPa]
        # g_p   :流量[m3/min]
        # pw_p  :消費電力[kW]
        # flag   :計算に問題があったら1、なかったら0
        # ef    :効率(0.0~1.0)
        self.pg = pg
        self.eg = eg
        self.r_ef = r_ef
        self.inv = 0
        self.dp = 0
        self.g = 0
        self.ef = 0
        self.pw = 0
        self.flag = 0
        self.num = 1
    
    def f2p(self, g): # flow to pressure for pump
        self.g = g
        # 流量がある場合のみ揚程を計算する
        if self.g > 0 and self.inv > 0:
            self.dp = (self.pg[0] + self.pg[1] *  (self.g / self.inv) + self.pg[2] *  (self.g / self.inv)**2) * self.inv**2
        else:
            self.dp = 0
            
        if self.dp < 0:
            self.dp = 0
            self.flag = 1
        else:
            self.flag = 0
            
        return self.dp
    
    def f2p_co(self): # coefficient for f2p
        return [self.pg[0]* self.inv**2, self.pg[1]*self.inv, self.pg[2]]
    
    
    def cal(self):
        # 流量がある場合のみ消費電力を計算する
        if self.g > 0 and self.inv > 0:
        
            # G: INV=1.0時（定格）の流量
            G = self.g / self.inv
            # K: 効率換算係数
            K = 1.0 - (1.0 - self.r_ef) / (self.inv**0.2) / self.r_ef 
            # ef: 効率
            self.ef = K * (self.eg[0] + self.eg[1] * G + self.eg[2] * G**2)
            
            self.dp = (self.pg[0] + self.pg[1] *  (self.g / self.inv) + self.pg[2] *  (self.g / self.inv)**2) * self.inv**2
            if self.dp < 0:
                self.dp = 0
                self.flag = 1
            # print(Nm)
        
            #  軸動力を求める
            if self.ef > 0:
                self.pw = 1.0 * self.g * self.dp / (60 * self.ef)
                self.flag = 0
            else:
                self.pw = 0
                self.flag = 2
            # print(self.flag)
        
        else:
            self.pw = 0.0
            self.flag = 0

ポンプのAPIドキュメント

In [ ]:
# Pump.f2p(pg=[233,5.9578,-4.95], inv=0, g)
# 流量を用いてポンプにより揚程を求める
# パラメータ : pq :圧力-流量(pg)曲線の係数（切片、一次、二次）
#             inv :回転数比(0.0~1.0)
#             g :流量
# 戻り値: dp :ポンプ揚程[kPa]
# Error Returns: flag = 1 (dp < 0) 
#                

# Pump.f2p_co=(pg=[233,5.9578,-4.95], inv=0)
# 流量を用いてポンプにより揚程を求める時の効率を計算する
# パラメータ : pq :圧力-流量(pg)曲線の係数（切片、一次、二次）
#             inv :回転数比(0.0~1.0)
# 戻り値: 揚程を求める時の効率


# Pump.cal(pg=[233,5.9578,-4.95], eg=[0.0099,0.4174,-0.0508], r_ef=0.8, inv=0,  g)
# 流量がある場合ポンプの消費電力を求める
# パラメータ : pq :圧力-流量(pg)曲線の係数（切片、一次、二次）
#             eq :効率-流量(eg)曲線の係数（切片、一次、二次）
#             r_ef  :定格時の最高効率 rated efficiency
#             inv :回転数比(0.0~1.0)
#             g :流量[m3/min]
# 戻り値: pw :消費電力[kW]
# Error Returns: flag = 1 (dp < 0) dp :ポンプ揚程[kPa]
#                flag = 2(ef < 0) ef: 効率

以下でポンプの設定値を入れる

In [ ]:
pump = Pump()
#以下で初期値を入れる
pump.pg = [233,5.9578,-4.95]
pump.inv = 0.5

In [ ]:
pump.g = 1

In [ ]:
print(pump.f2p(g))

冷却塔のソースコード

In [ ]:
# 冷却塔
class CoolingTower:
    def __init__(self, ua=143000, kr=1.0):
        # ua        :交換面積
        # g_w       :冷却水流量[kg/s]
        # g_a       :風量[kg/s]
        # tin_w     :冷却水入口温度[℃]
        # t_da      :外気乾球温度[℃]
        # rh        :外気相対湿度(0~100)
        # inv       :ファンインバーター周波数比（0.0~1.0）
        # flag      :収束計算確認のフラグ
        self.ua = ua
        self.kr = kr
        self.g_w = 0
        self.tin_w = 15
        self.g_a = 0
        self.t_da = 15
        self.rh = 50
        self.inv = 0
        self.flag = 0
        self.pw = 0
        self.dp = 0
        self.tout_w = 15
        
    def cal(self, g_w, Twin, Tda, rh):
        # cpw       :冷却水の比熱 [J/kg'C]
        # cp        :湿り空気（外気）の比熱 [J/kg'C]
        self.g_w = g_w
        self.Tda = Tda
        self.rh = rh
        self.tin_w = Twin
        self.g_a = self.inv*2000 # [m3/min]。ここで風量を与えてしまう
        self.pw = -35.469 * self.inv**3 + 59.499 * self.inv**2 - 1.6185 * self.inv + 0.7 #[kW]ここで計算してしまう
        if self.g_a < 10: # natural wind
            self.g_a = 10
            
        # 単位換算([m3/min] -> [kg/s])
        g_w = self.g_w * 10**3 / 60
        g_a = self.g_a * 1.293 / 60
        cpw = 4184
        cp = 1100
        
        if g_w > 0:
        
            # 乾球温度と湿球温度から外気比エンタルピーを求める
            [hin,xin] = enthalpy(Tda,rh)
            Twbin = tda_rh2twb(Tda,rh)
            
            # 湿球温度の飽和空気の比エンタルピーを求める
            # print(Twin,Twbin)
            # 空気出口湿球温度の最大値・最小値
            Twboutmax = max(Twin,Twbin);
            Twboutmin = min(Twin,Twbin);
            
            # 収束計算に入るための適当な初期値設定
            Twbout0 = 1;
            Twbout = 0;
            cnt = 0
            self.flag = 0
            
            while (Twbout0 - Twbout < - 0.01)or(Twbout0 - Twbout > 0.01):
    
                # 空気出口湿球温度の仮定
                Twbout0 = (Twboutmax + Twboutmin) / 2
                
                # 出口空気は飽和空気という仮定で、出口空気の比エンタルピーを求める。
                [hout, xout] = enthalpy(Twbout0,100)
                
                # 空気平均比熱cpeの計算
                dh = hout - hin
                dTwb = Twbout0 - Twbin
                cpe = dh / dTwb
            
                ua_e = self.ua * cpe / cp
                
                Cw = g_w * cpw
                Ca = g_a * cpe
                Cmin = min(Cw,Ca)
                Cmax = max(Cw,Ca)
                if Cmin == 0:      # 苦肉の策
                    Cmin = 0.001
                if Cmax == 0:
                    Cmax = 0.001
                    
                NTU = ua_e / Cmin
                
                eps = (1 - math.exp(-NTU * (1 - Cmin / Cmax))) / (1 - Cmin / Cmax * math.exp(-NTU * (1 - Cmin / Cmax)))
                
                Q = eps * Cmin * (Twin - Twbin)
                
                Twbout = Twbin + Q / Ca
            
                if Twbout < Twbout0:
                    Twboutmax = Twbout0
                else:
                    Twboutmin = Twbout0
                
                cnt += 1
                if cnt > 30:
                    self.flag = 1
                    break
            
            self.tout_w = Twin - Q / Cw
        
        self.dp = -self.kr*self.g_w**2
        
        return self.tout_w
    
    def f2p(self,g_w):
        self.g_w = g_w
        self.dp = -self.kr_w*self.g_w**2
        return self.dp
        
    def f2p_co(self):
        return [0,0,-self.kr]

# 比エンタルピーの算出
def enthalpy(Tda, rh):
# 乾球温度Tdaと相対湿度rhから比エンタルピーhと絶対湿度xを求める
# h     :enthalpy [J/kg]
# Tda   :Temperature of dry air ['C]
# rh    :relative humidity [%]
# cpd   :乾き空気の定圧比熱 [J / kg(DA)'C]
# gamma :飽和水蒸気の蒸発潜熱 [J/kg]
# cpv   :水蒸気の定圧比熱 [J / kg(DA)'C]
# x     :絶対湿度 [kg/kg(DA)]
    
    cpd = 1007
    gamma = 2499 * 10**3
    cpv = 1845
    
# 飽和水蒸気圧Ps[mmHg]の計算
    c = 373.16 / (273.16 + Tda)
    b = c - 1
    a = -7.90298 * b + 5.02808 * math.log10(c) - 1.3816 * 10**(-7) * (10**(11.344 * b / c) - 1) + 8.1328 * 10**(-3) * (10**(-3.49149 * b) - 1)
    Ps = 760 * 10**a
    
    x = 0.622 * (rh * Ps) / 100 / (760 - rh * Ps / 100)
    
    h = cpd * Tda + (gamma + cpv * Tda) * x
    
    return [h, x]

# 乾球温度から飽和水蒸気圧[hPa]
def tda2ps(tda):
    # ps:飽和水蒸気圧[hPa]
    # Wagner equation
    x = (1 - (tda+273.15)/647.3)
    ps = 221200*math.exp((-7.76451*x + 1.45838*x**1.5+-2.7758*x**3-1.23303*x**6)/(1-x))
    return ps

# 乾球温度と相対湿度から湿球温度['C]
def tda_rh2twb(tda, rh):
    # pv:水蒸気分圧[hPa]
    ps = tda2ps(tda)
    pv_1 = rh/100*ps
    pv_2 = -99999
    twb = 0
    twbmax = 50
    twbmin = -50
    cnt = 0
    while abs(pv_1-pv_2) > 0.01:
        twb = (twbmax + twbmin) / 2
        # Sprung equation
        pv_2 = tda2ps(twb)-0.000662*1013.25*(tda-twb)

        if pv_1-pv_2 > 0:
            twbmin = twb
        else:
            twbmax = twb
            
        cnt += 1
        if cnt > 20:
            break
            
    return twb

冷却塔のAPIドキュメント

In [ ]:
# CoolingTower.cal(g_w=0, Twin=15, Tda=15, rh=50, inv=0, g_a=10)
# 冷却塔における冷却水出口温度を求める
# パラメータ : g_w :冷却水流量[kg/s]
#             Twin :冷却水入口温度[℃]
#             Tda :外気乾球温度[℃]
#             rh  :外気相対湿度(0~100)
#             g_a :風量[kg/s]
# 戻り値: tout_w :冷却水出口温度[℃]
# Error Returns: flag = 1 (cnt > 30) 


# CoolingTower.f2p(g_w)
# 冷却水流量を用いて揚程を求める
# パラメータ : g_w :冷却水流量[kg/s]
# 戻り値: dp :冷却塔揚程[kPa]


# CoolingTower.f2p_co()
# 冷却水流量を用いて揚程を求める時の効率を計算する
# パラメータ : kr=1.0
# 戻り値: [0,0,-self.kr]

以下で冷却塔の設定値を入れる

In [ ]:
coolingtower = CoolingTower()
#以下で初期値を入れる
coolingtower.ua = 143000
coolingtower.kr = 1.0
coolingtower.inv = 0.5
coolingtower.g_a = 5

In [ ]:
g_w = 10
Twin = 25
Tda = 27
rh = 50

In [ ]:
print(coolingtower.cal(g_w, Twin, Tda, rh))

24.364029535193563


チラーのソースコード

In [ ]:
# 負荷率-COP曲線に基づく冷凍機COP計算。表は左から右、上から下に負荷率や冷却水入口温度が上昇しなければならない。
class Chiller:
    # 定格値の入力
    def __init__(self, spec_table=pd.read_csv("chiller_spec_table.csv",encoding="SHIFT-JIS",header=None)):
        # tin   :入口温度[℃]
        # tout  :出口温度[℃]
        # g     :流量[m3/min]
        # q     :熱量[kW]
        # ch    :冷水（chilled water）
        # cd    :冷却水(condenser water)
        # d     :定格値
        # pw    :消費電力[kW]  
        # lf    :部分負荷率(load factor, 0.0~1.0)
        # kr_ch :蒸発器圧力損失係数[kPa/(m3/min)**2]
        # kr_cd :凝縮器圧力損失係数[kPa/(m3/min)**2]
        # dp    :機器による圧力損失[kPa]
        self.tout_ch_d = float(spec_table.iat[1,0])
        self.tin_ch_d = float(spec_table.iat[1,1])
        self.g_ch_d = float(spec_table.iat[1,2])
        self.tin_cd_d = float(spec_table.iat[1,3])
        self.tout_cd_d = float(spec_table.iat[1,4])
        self.g_cd_d = float(spec_table.iat[1,5])
        # 定格冷凍容量[kW]
        self.q_ch_d = (self.tin_ch_d - self.tout_ch_d)*self.g_ch_d*1000*4.186/60
        # 定格主電動機入力[kW]
        self.pw_d = float(spec_table.iat[1,6])
        self.kr_ch = float(spec_table.iat[1,7])
        self.kr_cd = float(spec_table.iat[1,8])
        # 補機電力[kW]
        self.pw_sub = 0
        # 定格冷凍機COP
        self.COP_rp = self.q_ch_d / self.pw_d
        # 以下、毎時刻変わる可能性のある値
        self.tout_ch = 7
        self.tout_cd = 37
        self.pw = 0
        self.q_ch = 0
        self.lf = 0 # 負荷率(0.0~1.0)
        self.cop = 0
        self.flag = 0 #問題があったら1,なかったら0
        self.dp_ch = 0
        self.dp_cd = 0
        self.tin_cd = 15
        self.tin_ch = 7
        
        lf_cop = spec_table.drop(spec_table.index[[0, 1, 2]])
        lf_cop.iat[0,0] = '-'
        lf_cop = lf_cop.dropna(how='all', axis=1)
        self.data = lf_cop.values

        
    # 機器特性表に基づく冷凍機COPの算出
    def cal(self,tout_ch_sv, tin_ch, g_ch, tin_cd, g_cd):
        self.flag = 0
        self.tout_ch_sv = tout_ch_sv
        self.tin_ch = tin_ch
        self.g_ch = g_ch
        self.tin_cd = tin_cd
        self.g_cd = g_cd
        # 冷水出口温度[℃]
        self.tout_ch = self.tout_ch_sv
        # 冷凍熱量[kW]
        self.q_ch = (self.tin_ch - self.tout_ch)*self.g_ch*1000*4.186/60

        if self.q_ch > 0 and self.g_cd > 0:
             
            lf = self.data[0][1:].astype(np.float32)
            temp = self.data.transpose()[0][1:].astype(np.float32)
            dataset = self.data[1:].transpose()[1:].transpose().astype(np.float32)
            cop = RegularGridInterpolator((temp, lf), dataset)
            
            # 部分負荷率
            self.lf = self.q_ch / self.q_ch_d
            lf_cop = self.lf
            if self.lf > lf[-1]:
                self.lf = lf[-1]
                lf_cop = self.lf
                self.tout_ch += (self.q_ch - self.q_ch_d)/(self.g_ch*1000*4.186/60)
                self.q_ch = self.q_ch_d
                self.flag = 1

            elif self.lf < lf[0]:
                lf_cop = lf[-1]
                self.flag = 2
            
            tin_cd_cop = self.tin_cd-(self.tout_ch-self.tout_ch_d)
            
            if tin_cd_cop < temp[0]:
                tin_cd_cop = temp[0]
                self.flag = 3
            elif tin_cd_cop > temp[-1]:
                tin_cd_cop = temp[-1]
                self.flag = 4

            self.cop = float(cop([[tin_cd_cop, lf_cop]]))
            self.pw = self.q_ch / self.cop + self.pw_sub
            self.tout_cd = (self.q_ch + self.pw)/(4.186*self.g_cd*1000/60) + self.tin_cd
            
            
        elif self.q_ch == 0:
            self.pw = 0
            self.cop = 0
            # self.tout_cd = self.tin_cd
            self.flag = 0
        else:
            self.pw = 0
            self.cop = 0
            # self.tout_cd = self.tin_cd
            self.flag = 5
        
        self.dp_ch = -self.kr_ch*g_ch**2
        self.dp_cd = -self.kr_cd*g_cd**2

チラーのAPIドキュメント

In [ ]:
# Chiller.cal(tout_ch_sv, tin_ch, g_ch, tin_cd, g_cd)
# 機器特性表に基づく冷凍機COPを計算する
# パラメータ : tout_ch_sv :冷水出口温度[℃]
#              tin_ch :冷水入口温度[℃]
#              g_ch:冷水流量[m3/min]
#              tin_cd:冷却水入口温度[℃]
#              g_cd:冷却水流量[m3/min]
# 戻り値: cop :冷凍機COP
#         pw :消費電力[kW]  
#         dp_ch :機器による冷水圧力損失[kPa]
#         dp_cd :機器による冷却水圧力損失[kPa]        
# Error Returns: flag = 1

以下でチラーの設定値を入れる

In [ ]:
chiller = Chiller()
#以下で初期値を入れる
spec_table = pd.read_csv("chiller_spec_table.csv",encoding="SHIFT-JIS",header=None)

chiller.tout_ch_d = float(spec_table.iat[1,0])
chiller.tin_ch_d = float(spec_table.iat[1,1])
chiller.g_ch_d = float(spec_table.iat[1,2])
chiller.tin_cd_d = float(spec_table.iat[1,3])
chiller.tout_cd_d = float(spec_table.iat[1,4])
chiller.g_cd_d = float(spec_table.iat[1,5])
# 定格冷凍容量[kW]
chiller.q_ch_d = (chiller.tin_ch_d - chiller.tout_ch_d)*chiller.g_ch_d*1000*4.186/60
# 定格主電動機入力[kW]
chiller.pw_d = float(spec_table.iat[1,6])
chiller.kr_ch = float(spec_table.iat[1,7])
chiller.kr_cd = float(spec_table.iat[1,8])
# 補機電力[kW]
chiller.pw_sub = 0
# 定格冷凍機COP
chiller.COP_rp = chiller.q_ch_d / chiller.pw_d
chiller.lf = 0.5

In [ ]:
tout_ch_sv = 7
tin_ch = 7
g_ch = 10
tin_cd = 15
g_cd = 10

In [ ]:
print(chiller.cal(tout_ch_sv, tin_ch, g_ch, tin_cd, g_cd))